In [59]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


In [60]:
import utils

In [61]:
itp_id = 300

In [62]:
pd.set_option("display.max_columns", 100)


In [63]:
analysis_date_start = '2022-05-01'
analysis_date_end = '2022-05-31'

In [64]:
gtfs_daily = utils.load_schedule_data(analysis_date_start, analysis_date_end, itp_id)

In [65]:
gtfs_daily

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,-5118958034454309391,883656,3501,300,0,77710,2022-05-24,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,14,14,59340,60540,0.33
1,4100255323842349957,6797445014565992395,882488,3489,300,0,10,2022-05-16,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14,14,20580,21720,0.32
2,4100255323842349957,-1109410076312802103,882497,3489,300,0,10,2022-05-02,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14,14,34500,35940,0.40
3,4100255323842349957,-1416126210572631739,882561,3489,300,0,10,2022-05-12,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14,14,30180,31800,0.45
4,4100255323842349957,6289327327109149813,882524,3489,300,0,10,2022-05-19,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14,14,54660,56040,0.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39725,4100255323842349957,-4320983926927928772,881801,3485,300,0,2,2022-05-15,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,27780,31140,0.93
39726,4100255323842349957,4785778584599797020,881760,3485,300,0,2,2022-05-15,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,64860,68640,1.05
39727,4100255323842349957,6876715275502418648,881935,3485,300,0,10,2022-05-05,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,80100,82800,0.75
39728,4100255323842349957,-2137808411563612856,881943,3485,300,0,10,2022-05-24,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,71940,75060,0.87


In [66]:
gtfs_daily.service_date.min()

datetime.date(2022, 5, 1)

In [67]:
gtfs_daily.service_date.max()

datetime.date(2022, 5, 31)

In [68]:
rt = utils.load_rt_data(analysis_date_start, analysis_date_end)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [69]:
rt

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id
0,300,0,2022-05-01,882742,1301
1,300,0,2022-05-01,882736,1301
2,300,0,2022-05-01,882739,1301
3,300,0,2022-05-01,882771,1301
4,300,0,2022-05-01,882761,1301
...,...,...,...,...,...
38896,300,0,2022-05-31,881885,5316
38897,300,0,2022-05-31,883406,5316
38898,300,0,2022-05-31,883414,5316
38899,300,0,2022-05-31,883385,5316


In [70]:
rt['str_len'] = rt.trip_id.str.len()

In [71]:
rt.str_len.value_counts()

6    38901
Name: str_len, dtype: int64

In [72]:
len(rt)

38901

In [73]:
rt.trip_id.nunique()

3054

In [74]:
len(gtfs_daily)

39730

In [75]:
gtfs_daily.trip_id.nunique()

3158

In [76]:
date_list =['2022-05-01','2022-05-02', '2022-05-31']

In [77]:

from datetime import date, timedelta
start_date = date(2022, 5, 1)
end_date = date(2022, 5, 31)

In [78]:
# daterange = pd.date_range(start_date, end_date)

In [79]:
# (daterange)

In [80]:
#above range does not work with function

In [81]:
dates = set()

def daterange2(start, end):
    for n in range(int((end - start).days) + 1):
        yield start + timedelta(n)

for single_date in daterange2(start_date, end_date):
    dates.add(single_date.strftime('%Y-%m-%d'))

In [82]:
(dates)

{'2022-05-01',
 '2022-05-02',
 '2022-05-03',
 '2022-05-04',
 '2022-05-05',
 '2022-05-06',
 '2022-05-07',
 '2022-05-08',
 '2022-05-09',
 '2022-05-10',
 '2022-05-11',
 '2022-05-12',
 '2022-05-13',
 '2022-05-14',
 '2022-05-15',
 '2022-05-16',
 '2022-05-17',
 '2022-05-18',
 '2022-05-19',
 '2022-05-20',
 '2022-05-21',
 '2022-05-22',
 '2022-05-23',
 '2022-05-24',
 '2022-05-25',
 '2022-05-26',
 '2022-05-27',
 '2022-05-28',
 '2022-05-29',
 '2022-05-30',
 '2022-05-31'}

In [83]:
rt['date'] = pd.to_datetime(rt['date'])

In [84]:
gtfs_daily['service_date'] = pd.to_datetime(gtfs_daily['service_date'])

In [85]:
single_date2 = '2022-05-08'

(rt>>filter(_.date == single_date2))


,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id,str_len
8910,300,0,2022-05-08,882227,1303,6
8911,300,0,2022-05-08,882219,1303,6
8912,300,0,2022-05-08,882246,1303,6
8913,300,0,2022-05-08,882212,1303,6
8914,300,0,2022-05-08,882217,1303,6
...,...,...,...,...,...,...
9692,300,0,2022-05-08,881140,5317,6
9693,300,0,2022-05-08,881211,5317,6
9694,300,0,2022-05-08,881201,5317,6
9695,300,0,2022-05-08,881356,5317,6


In [86]:
# def get_pct_ran_df2(itp_id, list_of_dates):
#     pcts = []
#     for single_date in list_of_dates:
#         gtfs_daily2 = (gtfs_daily>>filter(_.service_date == single_date))
#         rt2 = (rt>>filter(_.date == single_date))
#         sched_rt_df = (pd.merge(gtfs_daily2, rt2, how='outer', on='trip_id', indicator='have_rt'))
#         #pct_ran = (utils.get_pct_ran(sched_rt_df, single_date))
#         day_pct_ran = {}
#         day_pct_ran['date'] = single_date
#         if ((len(sched_rt_df))!=0):
#             day_pct_ran['pct_trips_ran'] = ((len(sched_rt_df>>filter(_.have_rt=='both')))/(len(gtfs_daily2)))
#         elif ((len(sched_rt_df))==0):
#             day_pct_ran['pct_trips_ran'] = ''
#         pct_ran = pd.DataFrame([day_pct_ran])
#         pct_ran['n_have_rt'] = (len(sched_rt_df>>filter(_.have_rt=='both')))
#         pct_ran['n_missing_rt'] = (len(sched_rt_df>>filter(_.have_rt=='right_only')))
#         pct_ran['unmatched_rt'] = (len(sched_rt_df>>filter(_.have_rt=='left_only')))
#         pct_ran['nunique_sched'] = (gtfs_daily2.trip_id.nunique())
#         pct_ran['nunique_rt'] = (rt2.trip_id.nunique())

#         pcts.append(pct_ran)                                                    
#         #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
#     pcts = pd.concat(pcts)
#     pcts = pcts>>arrange(_.date)
#     return pd.DataFrame(pcts)


In [87]:
may = utils.get_pct_ran_df(itp_id, dates, gtfs_daily, rt)

In [88]:
may

,date,pct_trips_ran,n_have_rt,n_missing_rt,unmatched_rt,nunique_sched,nunique_rt
0,2022-05-01,0.68,527,252,244,770,778
0,2022-05-02,0.71,1069,252,450,1516,1318
0,2022-05-03,0.97,1473,0,151,1516,1365
0,2022-05-04,0.98,1487,0,135,1516,1381
0,2022-05-05,0.96,1455,0,173,1516,1343
0,2022-05-06,0.95,1392,0,169,1464,1295
0,2022-05-07,0.70,608,395,266,870,999
0,2022-05-08,0.65,504,283,269,770,783
0,2022-05-09,0.70,1060,247,463,1516,1300
0,2022-05-10,0.98,1482,0,136,1516,1380


In [89]:
import altair as alt
from dla_utils import _dla_utils

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [90]:
bar = (alt.Chart(may)
        .mark_bar()
        .encode(
            x=alt.X('date', title=_dla_utils.labeling('date'), sort=("x")),
            y=alt.Y('pct_trips_ran', title=_dla_utils.labeling('pct_trips_ran')),
            color=alt.Color(
                'pct_trips_ran',
                scale=alt.Scale(range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                legend=alt.Legend(title=(_dla_utils.labeling('pct_trips_ran')), symbolLimit=10)
            )
        ).properties(title='"Percent of Scheduled Trips with RT Data"'))
    
chart = styleguide.preset_chart_config(bar)
chart

alt.Chart(...)

In [93]:
base = alt.Chart(may).encode(x=alt.X('date:O', title=_dla_utils.labeling('date')))
# bar = (alt.Chart(may)
#         .mark_bar()
#         .encode(y=alt.Y('nunique_sched', title=_dla_utils.labeling('pct_trips_ran')),
#             color=alt.Color(
#                 'nunique_sched',
#                 scale=alt.Scale(range="#2EA8CE"),
#                 legend=alt.Legend(title=(_dla_utils.labeling('pct_trips_ran')), symbolLimit=10)
#             )
#         ))

bar = base.mark_bar().encode(y=alt.Y('nunique_sched:Q', title=_dla_utils.labeling('pct_trips_ran')))

line =  base.mark_line(color='#EB9F3C').encode(
    y='nunique_rt:Q'
)

(bar + line).properties(title="Percent of Scheduled Trips with RT Data")

alt.LayerChart(...)